In [ ]:
import torch

from src.model import get_model
from src.dataset import FrameDataset, split_dataset, get_pandas_dataset
from src.trainer import Trainer
from src.results import prepare_result

In [ ]:
baseTrainPath = 'data/train_images/train_images/'
train_dataset_path = 'data/usdc_train.json'

baseValPath = 'data/test_images/test_images/'
val_dataset_path = 'data/test_file_names.json'

In [ ]:
train_pandas_dataset, classes = get_pandas_dataset(train_dataset_path)

frame_dataset = FrameDataset(train_pandas_dataset, baseTrainPath)
train_dataset, test_dataset = split_dataset(frame_dataset)

pin_memory = torch.cuda.is_available()

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    collate_fn=lambda x: x,
    pin_memory=pin_memory)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    collate_fn=lambda x: x,
    pin_memory=pin_memory)

val_pandas_dataset, _ = get_pandas_dataset(val_dataset_path, val=True)
val_frame_dataset = FrameDataset(val_pandas_dataset, baseValPath, val=True)
val_index_map = list(val_pandas_dataset['id'])

val_dataloader = torch.utils.data.DataLoader(
    val_frame_dataset,
    batch_size=16,
    collate_fn=lambda x: x,
    pin_memory=pin_memory)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
autocast = torch.cuda.is_available()

model = get_model(len(classes)).to(device)
optimizer = torch.optim.SGD(filter(lambda param: param.requires_grad, model.parameters()), lr=0.001, momentum=0.9, weight_decay=0.0005)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.1, step_size_up=5, mode="exp_range", gamma=0.85)
num_epochs = 30

trainer = Trainer(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    val_dataloader=val_dataloader,
    optimizer=optimizer,
    scheduler=scheduler,
    num_epochs=num_epochs,
    device=device,
    autocast=autocast
)

trainer.train()

result = trainer.eval()
prepare_result(result, 'result.csv', val_index_map)